In [14]:
text="""
The Ring and the Road
In lands where shadows twist and creep,
Where mountains brood and rivers sleep,
A whisper stirs of power old—
A ring of fire, a band of gold.

Forged by flame in Mordor's night,
It seeks the dark, it shuns the light.
One ring to rule, to bind, to bring
All under shadow's smothering wing.

From gentle Shire where hobbits dwell,
A tale begins, too vast to tell.
With Frodo, bearer frail but true,
And Sam, whose heart was brave and blue.

Through ancient woods and haunted vale,
O'er snowy peaks and paths grown pale,
They marched with elf, with dwarf, with man,
United in a broken plan.

Aragorn with kingly grace,
Legolas swift in deadly race,
Gimli, stout with axe in hand,
They swore to free a sundered land.

Gandalf rose with fire and word,
Through Balrog’s flame his soul was stirred.
Returned anew in robes of white,
To turn the tide in darkest fight.

And Gollum crept with poisoned mind,
Twisted fate and truth entwined.
Guide and traitor, slave and thief,
He bore the weight of endless grief.

In towers tall and caverns deep,
In halls where even ghosts might weep,
They battled death and time and fear,
To cast the ring in fire sear.

Till Mount Doom’s fire did wane and cry,
And Sauron fell with one last eye.
The eagles soared, the darkness broke,
The world awoke in hope and oak.

So sing, O bards, of Elven grace,
Of Dúnedain and Entish face.
Of hobbits bold and battles grim,
Of stars that shone when days were dim.

For in the end, when all was lost,
The smallest hearts paid dearest cost.
And though the Ring has passed away,
Its echoes stir the soul today.
"""
text = text.split(' ')
#grouped each 10 words together
text = [text[i:i+20] for i in range(0, len(text), 10)]
#join each group with a space
text = [' '.join(group) for group in text]
#join all groups with a newline
# text = '\n'.join(text)
text


['\nThe Ring and the Road\nIn lands where shadows twist and creep,\nWhere mountains brood and rivers sleep,\nA whisper stirs of power',
 'creep,\nWhere mountains brood and rivers sleep,\nA whisper stirs of power old—\nA ring of fire, a band of gold.\n\nForged by flame',
 "old—\nA ring of fire, a band of gold.\n\nForged by flame in Mordor's night,\nIt seeks the dark, it shuns the light.\nOne",
 "in Mordor's night,\nIt seeks the dark, it shuns the light.\nOne ring to rule, to bind, to bring\nAll under shadow's smothering",
 "ring to rule, to bind, to bring\nAll under shadow's smothering wing.\n\nFrom gentle Shire where hobbits dwell,\nA tale begins, too vast",
 'wing.\n\nFrom gentle Shire where hobbits dwell,\nA tale begins, too vast to tell.\nWith Frodo, bearer frail but true,\nAnd Sam, whose heart',
 "to tell.\nWith Frodo, bearer frail but true,\nAnd Sam, whose heart was brave and blue.\n\nThrough ancient woods and haunted vale,\nO'er snowy",
 "was brave and blue.\n\nThrough ancient wo

In [15]:
from openai import OpenAI
client = OpenAI()
def get_openai_response(text):
    completion = client.chat.completions.create(
        model="o4-mini-2025-04-16",
        messages=[
            {
                "role": "user",
                "content": f"Translate this text to Hebrew: {text} just return the translation without any additional text."
            }
        ]
    )

    return completion.choices[0].message.content

In [16]:
for i in range(0,10):
    word = get_openai_response(text[i])
    print(text[i],word)


The Ring and the Road
In lands where shadows twist and creep,
Where mountains brood and rivers sleep,
A whisper stirs of power הטבעת והדרך

בארצות שבהן צללים מתפתלים ומזחילים,  
שבהן ההרים מאיימים והנהרות ישנים,  
לחישה מעוררת של כוח
creep,
Where mountains brood and rivers sleep,
A whisper stirs of power old—
A ring of fire, a band of gold.

Forged by flame הִזְחַל,  
שָׁם הָהָרִים מִתְבּוֹנְנִים וְהַנְּהָרוֹת יְשֵׁנוֹת,  
לַחַשׁ מֵעוֹרֵר שֶׁל כֹּחַ עַתִּיק—  
טַבְעַת אֵשׁ, חַגּוֹרַת זָהָב.  
יצוקה בלהבה.
old—
A ring of fire, a band of gold.

Forged by flame in Mordor's night,
It seeks the dark, it shuns the light.
One ישן—
טבעת אש, חישוק זהב.

יצוקה בלהבה בליל מורדור,  
היא מחפשת את החושך, ומתנכרת לאור.
in Mordor's night,
It seeks the dark, it shuns the light.
One ring to rule, to bind, to bring
All under shadow's smothering בליל מורדור  
מבקש הוא את החושך, דוחה הוא את האור.  
טבעת אחת למשול, לאסור, להביא  
את כולם תחת חניקת הצללים
ring to rule, to bind, to bring
All under shadow's s